In [1]:
# nuclio: ignore
import nuclio 

In [2]:
%nuclio config kind="nuclio:serving"
%nuclio env MODEL_CLASS=ChurnModel

%nuclio config spec.build.baseImage = "mlrun/ml-models"

%nuclio: setting kind to 'nuclio:serving'
%nuclio: setting 'MODEL_CLASS' environment variable
%nuclio: setting spec.build.baseImage to 'mlrun/ml-models'


In [3]:
%%nuclio cmd -c
python -m pip install lifelines

## Function Code

In [4]:
import os
import json
import numpy as np
import xgboost as xgb
from cloudpickle import load
from lifelines import CoxPHFitter, KaplanMeierFitter
import xgboost as xgb

### Model Serving Class

import mlrun
class ChurnModel(mlrun.runtimes.MLModelServer):
    def load(self):
        """
        load multiple models in nested folders, churn model only
        """
        clf_model_file, extra_data = self.get_model(".pkl")
        self.model = load(open(str(clf_model_file), "rb"))
        
        if "cox" in extra_data.keys():
            cox_model_file = extra_data["cox"]
            self.cox_model = load(open(str(cox_model_file), "rb"))
            if "cox/km" in extra_data.keys():
                km_model_file = extra_data["cox/km"]
                self.km_model = load(open(str(km_model_file), "rb"))
        return

    def predict(self, body):
        try:
            # we have potentially 3 models to work with:
            #if hasattr(self, "cox_model") and hasattr(self, "km_model"):
                # hack for now, just predict using one:
            feats = np.asarray(body["instances"], dtype=np.float32).reshape(-1, 23)
            result = self.model.predict(feats, validate_features=False)
            return result.tolist()
            #else:
            #    raise Exception("models not found")
        except Exception as e:
            raise Exception("Failed to predict %s" % e)

The following end-code annotation tells ```nuclio``` to stop parsing the notebook from this cell. _**Please do not remove this cell**_:

In [5]:
# nuclio: end-code

### mlconfig

In [6]:
from mlrun import mlconf
import os
mlconf.dbpath = mlconf.dbpath or "http://mlrun-api:8080"
mlconf.artifact_path = mlconf.artifact_path or f"{os.environ['HOME']}/artifacts"

<a id="test-locally"></a>
## Test the function locally

The class above can be tested locally. Just instantiate the class, `.load()` will load the model to a local dir.

> **Verify there is a model file in the model_dir path (generated by the training notebook)**

In [22]:
model_dir = os.path.join(mlconf.artifact_path, "churn", "models")

my_server = ChurnModel("my-model", model_dir=model_dir)
my_server.load()

In [33]:
import pandas as pd
xtest = pd.read_csv(os.path.join(mlconf.artifact_path, "churn", "test-set.csv"))

We can use the `.predict(body)` method to test the model.

In [34]:
import json, numpy as np

# this should fail if the churn model hasn't been saved properly
preds = my_server.predict({"instances":xtest.values[:10,:-1].tolist()})

In [35]:
print("predicted class:", preds)

predicted class: [0, 1, 0, 1, 1, 1, 0, 0, 0, 0]


<a id="deploy"></a>
### **deploy our serving class using as a serverless function**
in the following section we create a new model serving function which wraps our class , and specify model and other resources.

the `models` dict store model names and the assosiated model **dir** URL (the URL can start with `S3://` and other blob store options), the faster way is to use a shared file volume, we use `.apply(mount_v3io())` to attach a v3io (iguazio data fabric) volume to our function. By default v3io will mount the current user home into the `\User` function path.

**verify the model dir does contain a valid `model.bst` file**

In [36]:
from mlrun import new_model_server, mount_v3io
import requests

In [37]:
fn = new_model_server("churn-test",
                      model_class="ChurnModel",
                      models={"churn_server_v1": f"{model_dir}"})
fn.spec.description = "churn classification and predictor"
fn.metadata.categories = ["serving", "ml"]
fn.metadata.labels = {"author": "yashab", "framework": "churn"}

fn.export("function.yaml")

[mlrun] 2020-05-12 22:00:01,735 function spec saved to path: function.yaml


## tests

In [38]:
if "V3IO_HOME" in list(os.environ):
    from mlrun import mount_v3io
    fn.apply(mount_v3io())
else:
    # is you set up mlrun using the instructions at
    # https://github.com/mlrun/mlrun/blob/master/hack/local/README.md
    from mlrun.platforms import mount_pvc
    fn.apply(mount_pvc("nfsvol", "nfsvol", "/home/joyan/data"))

In [39]:
addr = fn.deploy(project="churn-project")

[mlrun] 2020-05-12 22:00:05,445 deploy started
[nuclio] 2020-05-12 22:00:09,560 (info) Build complete
[nuclio] 2020-05-12 22:00:23,745 (info) Function deploy complete
[nuclio] 2020-05-12 22:00:23,751 done updating churn-project-churn-test, function address: 3.21.117.61:30144


<a id="test-model-server"></a>
### **test our model server using HTTP request**


We invoke our model serving function using test data, the data vector is specified in the `instances` attribute.

In [40]:
# KFServing protocol event
event_data = {"instances": xtest.values[:10,:-1].tolist()}

In [41]:
import json
resp = requests.put(addr + "/churn_server_v1/predict", json=json.dumps(event_data))

# mlutils function for this?
tl = resp.text.replace("[","").replace("]","").split(",")
assert preds == [int(i) for i in np.asarray(tl)]

**[back to top](#top)**